<font size=8>&nbsp;<font color=#F7C5A1>**XDCClab =**</font> <font color=#F9AB00>**Colab**<font color=#C9E9FF> **+**<font color=#908672> **xdcc-dl**<font color=#C9E9FF> **+** <font color=#017547>**xdccget**

In case you stumbled upon this colab notebook somewhere on the internet, here's the **[repo](https://github.com/hyPnOtICDo0g/Otakulab)**.

## Mount Google Drive
To Download files we need to mount Google Drive.


In [ ]:
#@markdown #### 🔼 This cell will mount your Google Drive to /content/drive
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Google_Drive_icon_%282020%29.svg/672px-Google_Drive_icon_%282020%29.svg.png' height="50"></center>
#@markdown #### • Type the name of the Shared/Team drive if you have one (Optional)

from google.colab import drive
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]

#Option to directly download to your TD so as to remove the file size limits
#Names are case-sensitive
TD = ""  #@param {type:"string"}

#Enter your authorization code & click enter
if MODE == "MOUNT":
  drive.mount('/content/drive')
elif MODE == "UNMOUNT":
   drive.flush_and_unmount()

## Configuring the IRC clients
*IRC gang, where you at?*

In [ ]:
from google.colab import output
from os import path
import pathlib, time
if path.exists("/bin/xdccget") == False:
  !echo "Performing Shell Sorcery..."
  !pip install xdcc-dl -q > nul
  !git clone --quiet https://github.com/RhythmLunatic/xdccget.git
  output.clear()
  !cd xdccget;make -s;mv xdccget /bin
  !rm -rf sample_data nul xdccget
!echo "Configuring Stuff..."

#@markdown #### • You can either set a custom channel/server or select one from the drop down list.
#@markdown #### • You are need to run this cell again on making any changes below.

DL_DIR = "Downloads" #@param {type:"string"}
DL_LOC = "COLAB VM (70GB)" #@param ["COLAB VM (70GB)", "MY DRIVE (15GB)", "SHARED DRIVE (∞)"]
channel = '#nibl' #@param ["#nibl", "#MK", "#XDCCLeech"] {allow-input: true}
server = 'irc.rizon.net' #@param ["irc.rizon.net", "irc.xertion.org"] {allow-input: true}

if DL_LOC == "COLAB VM (70GB)":
  pathlib.Path("/content/"+DL_DIR).mkdir(parents=False, exist_ok=True)
  finloc = "/content/"+DL_DIR

#In case you skipped the above cell & didn't mount your drive or typed the TD's name, you'll get an error.
#You're trying to download to a folder which doesn't exist. ¯\_(ツ)_/¯
#Mount your drive by running the above cell & run this cell again
elif DL_LOC == "MY DRIVE (15GB)":
  pathlib.Path("/content/drive/MyDrive/"+DL_DIR).mkdir(parents=False, exist_ok=True)
  finloc = "/content/drive/MyDrive/"+DL_DIR
elif DL_LOC == "SHARED DRIVE (\u221E)":
  pathlib.Path("/content/drive/Shareddrives/"+TD+"/"+DL_DIR).mkdir(parents=False, exist_ok=True)
  finloc = "/content/drive/Shareddrives/"+TD+"/"+DL_DIR
time.sleep(8)
output.clear()

!echo "Done!"

## Using the required client to download
*Feeling nostalgic yet?*

In [ ]:
#@markdown #### • xdcc-dl will be set as default as it's stable & works great for batch downloads.
#@markdown #### • xdccget is more verbose, i.e. it includes a speed indicator, queue position, etc but no batch support.

client = "xdcc-dl" #@param ["xdcc-dl", "xdccget"]
command = "" #@param {type:"string"}

#The '.ipynb_checkpoints' folder can be deleted after the downloads are complete
#xdcc-dl tends to give an error when a download a forcefully interrupted, this can be safely ignored
if client == "xdcc-dl":
  cmdedit = command.replace("batch ", "send #")
  !cd "$finloc";xdcc-dl -v "$cmdedit" -s "$server" --fallback-channel "$channel"
elif client == "xdccget":
  bot = command.split()[1]
  pack = command.split()[4]
  !xdccget -v -i "$server" "$channel" "$bot xdcc send $pack" --dont-confirm-offsets -d "$finloc"

## The Rclone Wars
*Pew Pew* those downloaded files to your drive(s).

In [ ]:
#@markdown #### • Enter the source & destination paths & run this cell.
#@markdown #### • Upload your `rclone.conf` file when prompted.
#@markdown #### • You can get the **source path** by using the file manager to your left.
#@markdown #### • **Destination path** is the remote you wanna transfer to.

from os import path
from google.colab import output
from google.colab import files

if path.exists("/root/.config/rclone/rclone.conf") == False:

  files.upload()
  !mkdir -p /root/.config/rclone;mv rclone.conf /root/.config/rclone
  !curl -s https://rclone.org/install.sh | sudo bash > log
  output.clear()

dry_run = False #@param {type:"boolean"}
Source = "" #@param {type:"string"}
Destination = "" #@param {type:"string"}
Extra_args = "" #@param {type:"string"}
  
args = "--stats-one-line --fast-list -v "
if dry_run == True:
  args +="--dry-run "
if Extra_args != "":
  args +=Extra_args
!rclone copy "$Source" "$Destination" $args 